In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [ ]:
image = tf.image.decode_jpeg(tf.read_file('../datas/sample.jpg'), channels=3)
image = tf.cast( tf.image.resize_images(image, 32, 32), tf.float32 )

images = tf.train.batch([image], 1)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(1, figsize=(3, 3))
plt.imshow( image.eval(), interpolation='none' )



In [ ]:
from image_classification import ImageClassification

In [ ]:
#sess = tf.InteractiveSession()



nn = ImageClassification({}, sess)
output = nn.forward(images)

sess.run(tf.initialize_all_variables())
print output.eval()
